In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
prev = pd.read_csv('./data/rawdata/previous_application.csv.zip',compression='zip')
prev = prev.sort_values(['SK_ID_CURR','SK_ID_PREV'], ascending = [True,False])
prev['cnt'] = 1

In [3]:
prev.shape
prev.head()

(1670214, 38)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,cnt
201668,1369693,100001,Consumer loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,...,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0,1
892077,1038818,100002,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,...,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,1
1021650,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,...,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0,1
1223745,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,...,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0,1
575941,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,...,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0,1


In [4]:
# previous other source data
prev_cash_features = pd.read_csv('./data/rawdata/prev_cash_features.csv')
prev_cash_features = prev_cash_features.drop('SK_ID_CURR',axis = 1)
prev_cash_features.columns = ['cash_' + f for f in prev_cash_features.columns]

prev_cash_features.shape
prev_cash_features.columns
prev_cash_features.head()


(936325, 18)

Index(['cash_SK_ID_PREV', 'cash_MONTHS_BALANCE_min', 'cash_MONTHS_BALANCE_max',
       'cash_MONTHS_BALANCE_cnt', 'cash_CNT_INSTALMENT_min',
       'cash_CNT_INSTALMENT_max', 'cash_CNT_INSTALMENT_set',
       'cash_CNT_INSTALMENT_FUTURE_min', 'cash_CNT_INSTALMENT_FUTURE_max',
       'cash_NAME_CONTRACT_STATUS_cntd', 'cash_NAME_CONTRACT_STATUS_set',
       'cash_NAME_CONTRACT_STATUS_latest', 'cash_SK_DPD_max',
       'cash_SK_DPD_cnt0', 'cash_SK_DPD_DEF_max', 'cash_SK_DPD_DEF_cnt0',
       'cash_SK_DPD_diff_max', 'cash_SK_DPD_diff_cnt0'],
      dtype='object')

,cash_SK_ID_PREV,cash_MONTHS_BALANCE_min,cash_MONTHS_BALANCE_max,cash_MONTHS_BALANCE_cnt,cash_CNT_INSTALMENT_min,cash_CNT_INSTALMENT_max,cash_CNT_INSTALMENT_set,cash_CNT_INSTALMENT_FUTURE_min,cash_CNT_INSTALMENT_FUTURE_max,cash_NAME_CONTRACT_STATUS_cntd,cash_NAME_CONTRACT_STATUS_set,cash_NAME_CONTRACT_STATUS_latest,cash_SK_DPD_max,cash_SK_DPD_cnt0,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_cnt0,cash_SK_DPD_diff_max,cash_SK_DPD_diff_cnt0
0,1000001,-10,-8,3,2.0,12.0,"12.0,2.0",0.0,12.0,2,"Active,Completed",Completed,0,0,0,0,0,0
1,1000002,-54,-50,5,4.0,6.0,"6.0,4.0",0.0,4.0,2,"Active,Completed",Completed,0,0,0,0,0,0
2,1000003,-4,-1,4,12.0,12.0,12.0,9.0,12.0,1,Active,Active,0,0,0,0,0,0
3,1000004,-29,-22,8,7.0,10.0,"7.0,10.0",0.0,10.0,2,"Active,Completed",Completed,0,0,0,0,0,0
4,1000005,-56,-46,11,10.0,10.0,10.0,0.0,10.0,2,"Active,Completed",Completed,0,0,0,0,0,0


In [5]:
prev_cash_features.head()

,cash_SK_ID_PREV,cash_MONTHS_BALANCE_min,cash_MONTHS_BALANCE_max,cash_MONTHS_BALANCE_cnt,cash_CNT_INSTALMENT_min,cash_CNT_INSTALMENT_max,cash_CNT_INSTALMENT_set,cash_CNT_INSTALMENT_FUTURE_min,cash_CNT_INSTALMENT_FUTURE_max,cash_NAME_CONTRACT_STATUS_cntd,cash_NAME_CONTRACT_STATUS_set,cash_NAME_CONTRACT_STATUS_latest,cash_SK_DPD_max,cash_SK_DPD_cnt0,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_cnt0,cash_SK_DPD_diff_max,cash_SK_DPD_diff_cnt0
0,1000001,-10,-8,3,2.0,12.0,"12.0,2.0",0.0,12.0,2,"Active,Completed",Completed,0,0,0,0,0,0
1,1000002,-54,-50,5,4.0,6.0,"6.0,4.0",0.0,4.0,2,"Active,Completed",Completed,0,0,0,0,0,0
2,1000003,-4,-1,4,12.0,12.0,12.0,9.0,12.0,1,Active,Active,0,0,0,0,0,0
3,1000004,-29,-22,8,7.0,10.0,"7.0,10.0",0.0,10.0,2,"Active,Completed",Completed,0,0,0,0,0,0
4,1000005,-56,-46,11,10.0,10.0,10.0,0.0,10.0,2,"Active,Completed",Completed,0,0,0,0,0,0


In [6]:
prev = prev.merge(prev_cash_features, how = 'left', left_on = 'SK_ID_PREV',right_on = 'cash_SK_ID_PREV')
prev.shape
prev.head()

(1670214, 56)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,cash_CNT_INSTALMENT_FUTURE_max,cash_NAME_CONTRACT_STATUS_cntd,cash_NAME_CONTRACT_STATUS_set,cash_NAME_CONTRACT_STATUS_latest,cash_SK_DPD_max,cash_SK_DPD_cnt0,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_cnt0,cash_SK_DPD_diff_max,cash_SK_DPD_diff_cnt0
0,1369693,100001,Consumer loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,...,4.0,2.0,"Active,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0
1,1038818,100002,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,...,24.0,1.0,Active,Active,0.0,0.0,0.0,0.0,0.0,0.0
2,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,...,6.0,2.0,"Active,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0
3,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,...,12.0,1.0,Active,Active,0.0,0.0,0.0,0.0,0.0,0.0
4,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,...,12.0,2.0,"Active,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# previous other source data
prev_credit_features = pd.read_csv('./data/rawdata/prev_credit_features.csv')
prev_credit_features = prev_credit_features.drop('SK_ID_CURR',axis = 1)
prev_credit_features.columns = ['credit_' + f for f in prev_credit_features.columns]


prev_credit_features.shape
prev_credit_features.columns
prev_credit_features.head()

(104307, 72)

Index(['credit_SK_ID_PREV', 'credit_MONTHS_BALANCE_min',
       'credit_MONTHS_BALANCE_max', 'credit_MONTHS_BALANCE_count',
       'credit_AMT_BALANCE_min', 'credit_AMT_BALANCE_max',
       'credit_AMT_BALANCE_sum', 'credit_AMT_BALANCE_mean',
       'credit_AMT_BALANCE_size', 'credit_AMT_CREDIT_LIMIT_ACTUAL_min',
       'credit_AMT_CREDIT_LIMIT_ACTUAL_max',
       'credit_AMT_DRAWINGS_ATM_CURRENT_min',
       'credit_AMT_DRAWINGS_ATM_CURRENT_size',
       'credit_AMT_DRAWINGS_ATM_CURRENT_mean',
       'credit_AMT_DRAWINGS_ATM_CURRENT_sum',
       'credit_AMT_DRAWINGS_ATM_CURRENT_max',
       'credit_AMT_DRAWINGS_CURRENT_min', 'credit_AMT_DRAWINGS_CURRENT_size',
       'credit_AMT_DRAWINGS_CURRENT_mean', 'credit_AMT_DRAWINGS_CURRENT_sum',
       'credit_AMT_DRAWINGS_CURRENT_max',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_min',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_size',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_mean',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_sum',
       'credit_A

,credit_SK_ID_PREV,credit_MONTHS_BALANCE_min,credit_MONTHS_BALANCE_max,credit_MONTHS_BALANCE_count,credit_AMT_BALANCE_min,credit_AMT_BALANCE_max,credit_AMT_BALANCE_sum,credit_AMT_BALANCE_mean,credit_AMT_BALANCE_size,credit_AMT_CREDIT_LIMIT_ACTUAL_min,...,credit_CNT_INSTALMENT_MATURE_CUM_set,credit_CNT_INSTALMENT_MATURE_CUM_latest,credit_NAME_CONTRACT_STATUS_cntd,credit_NAME_CONTRACT_STATUS_set,credit_SK_DPD_max,credit_SK_DPD_size,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_size,credit_SK_DPD_diff_max,credit_SK_DPD_diff_size
0,1000018,-6,-2,5,38879.145,136695.420,374731.425,74946.285000,5.0,45000.0,...,"3.0,2.0,0.0,1.0,4.0",4.0,1,Active,0,0,0,0,0,0
1,1000030,-8,-1,8,15583.635,103027.275,447928.515,63989.787857,7.0,45000.0,...,"5.0,3.0,2.0,0.0,1.0,4.0",5.0,1,Active,0,0,0,0,0,0
2,1000031,-16,-1,16,5805.495,154945.935,838311.030,104788.878750,8.0,45000.0,...,"5.0,3.0,10.0,9.0,2.0,0.0,7.0,8.0,6.0,1.0,4.0",10.0,1,Active,0,0,0,0,0,0
3,1000035,-6,-2,5,NaN,0.000,0.000,NaN,0.0,225000.0,...,0.0,0.0,1,Active,0,0,0,0,0,0
4,1000077,-12,-2,11,NaN,0.000,0.000,NaN,0.0,45000.0,...,0.0,0.0,1,Active,0,0,0,0,0,0


In [8]:
prev = prev.merge(prev_credit_features, how = 'left', left_on = 'SK_ID_PREV',right_on = 'credit_SK_ID_PREV')
prev.shape
prev.head()

(1670214, 128)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,credit_CNT_INSTALMENT_MATURE_CUM_set,credit_CNT_INSTALMENT_MATURE_CUM_latest,credit_NAME_CONTRACT_STATUS_cntd,credit_NAME_CONTRACT_STATUS_set,credit_SK_DPD_max,credit_SK_DPD_size,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_size,credit_SK_DPD_diff_max,credit_SK_DPD_diff_size
0,1369693,100001,Consumer loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1038818,100002,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# previous other source data
prev_installment_features = pd.read_csv('./data/rawdata/prev_installment_features.csv')
prev_installment_features.columns = ['installment_' + f for f in prev_installment_features.columns]


prev_installment_features.shape
prev_installment_features.columns
prev_installment_features.head()

(997752, 45)

Index(['installment_SK_ID_PREV', 'installment_NUM_INSTALMENT_VERSION_nunique',
       'installment_NUM_INSTALMENT_VERSION_set',
       'installment_NUM_INSTALMENT_VERSION_latest',
       'installment_NUM_INSTALMENT_NUMBER_min',
       'installment_NUM_INSTALMENT_NUMBER_max',
       'installment_NUM_INSTALMENT_NUMBER_count',
       'installment_NUM_INSTALMENT_NUMBER_nunique',
       'installment_DAYS_INSTALMENT_min', 'installment_DAYS_INSTALMENT_max',
       'installment_DAYS_INSTALMENT_median',
       'installment_DAYS_INSTALMENT_mean', 'installment_DAYS_INSTALMENT_range',
       'installment_DAYS_ENTRY_PAYMENT_min',
       'installment_DAYS_ENTRY_PAYMENT_max',
       'installment_DAYS_ENTRY_PAYMENT_median',
       'installment_DAYS_ENTRY_PAYMENT_mean',
       'installment_DAYS_ENTRY_PAYMENT_range', 'installment_DPD_min',
       'installment_DPD_max', 'installment_DPD_median', 'installment_DPD_mean',
       'installment_DPD_sum', 'installment_DPD_std',
       'installment_AMT_INSTALMEN

,installment_SK_ID_PREV,installment_NUM_INSTALMENT_VERSION_nunique,installment_NUM_INSTALMENT_VERSION_set,installment_NUM_INSTALMENT_VERSION_latest,installment_NUM_INSTALMENT_NUMBER_min,installment_NUM_INSTALMENT_NUMBER_max,installment_NUM_INSTALMENT_NUMBER_count,installment_NUM_INSTALMENT_NUMBER_nunique,installment_DAYS_INSTALMENT_min,installment_DAYS_INSTALMENT_max,...,installment_AMT_PAYMENT_median,installment_AMT_PAYMENT_std,installment_AMT_PAYMENT_nunique,installment_AMT_DPD_min,installment_AMT_DPD_max,installment_AMT_DPD_mean,installment_AMT_DPD_sum,installment_AMT_DPD_median,installment_AMT_DPD_std,installment_AMT_DPD_nunique
0,1000001,2,"2.0,1.0",1.0,1,2,2,2,-268.0,-238.0,...,34221.7125,39339.747885,2,0.0,0.000,0.000000,0.000,0.0,0.000000,1
1,1000002,2,"2.0,1.0",1.0,1,4,4,4,-1600.0,-1510.0,...,6264.0000,6089.782500,2,0.0,0.000,0.000000,0.000,0.0,0.000000,1
2,1000003,1,1.0,1.0,1,3,3,3,-94.0,-34.0,...,4951.3500,0.000000,1,0.0,0.000,0.000000,0.000,0.0,0.000000,1
3,1000004,2,"2.0,1.0",1.0,1,7,7,7,-862.0,-682.0,...,3391.1100,3698.527885,2,0.0,0.000,0.000000,0.000,0.0,0.000000,1
4,1000005,1,1.0,1.0,1,10,11,10,-1688.0,-1418.0,...,14713.6050,4432.077970,4,0.0,14710.815,1337.600455,14713.605,0.0,4435.393568,3


In [10]:
prev = prev.merge(prev_installment_features, how = 'left', left_on = 'SK_ID_PREV',right_on = 'installment_SK_ID_PREV')
prev.shape
prev.head()

(1670214, 173)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,installment_AMT_PAYMENT_median,installment_AMT_PAYMENT_std,installment_AMT_PAYMENT_nunique,installment_AMT_DPD_min,installment_AMT_DPD_max,installment_AMT_DPD_mean,installment_AMT_DPD_sum,installment_AMT_DPD_median,installment_AMT_DPD_std,installment_AMT_DPD_nunique
0,1369693,100001,Consumer loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,...,3951.000,6723.450000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1038818,100002,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,...,9251.775,10058.037722,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,...,64567.665,119.228913,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,...,6737.310,21.460110,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,...,98356.995,174800.391505,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
features_type = {
'AMT_ANNUITY':'amount_features', 'AMT_APPLICATION':'amount_features', 'AMT_CREDIT':'amount_features', 'AMT_GOODS_PRICE':'amount_features', 'AMT_DOWN_PAYMENT':'amount_features', 'NAME_CONTRACT_TYPE':'numerical_features', 'RATE_DOWN_PAYMENT':'numerical_features', 'RATE_INTEREST_PRIMARY':'numerical_features', 'RATE_INTEREST_PRIVILEGED':'numerical_features', 'DAYS_DECISION':'numerical_features', 'SELLERPLACE_AREA':'numerical_features', 'DAYS_FIRST_DRAWING':'numerical_features', 'DAYS_FIRST_DUE':'numerical_features', 'DAYS_LAST_DUE_1ST_VERSION':'numerical_features', 'DAYS_LAST_DUE':'numerical_features', 'DAYS_TERMINATION':'numerical_features', 'WEEKDAY_APPR_PROCESS_START':'categorical_features', 'HOUR_APPR_PROCESS_START':'categorical_features', 'NAME_CASH_LOAN_PURPOSE':'categorical_features', 'NAME_GOODS_CATEGORY':'categorical_features', 'NAME_CONTRACT_STATUS':'pivot_features', 'NAME_PAYMENT_TYPE':'pivot_features', 'CODE_REJECT_REASON':'pivot_features', 'NAME_TYPE_SUITE':'pivot_features', 'NAME_CLIENT_TYPE':'pivot_features', 'NAME_PORTFOLIO':'pivot_features', 'NAME_PRODUCT_TYPE':'pivot_features', 'CHANNEL_TYPE':'pivot_features', 'NAME_SELLER_INDUSTRY':'pivot_features', 'NAME_YIELD_GROUP':'pivot_features', 'PRODUCT_COMBINATION':'pivot_features', 
'cash_MONTHS_BALANCE_min':'numerical_features', 'cash_MONTHS_BALANCE_max':'numerical_features', 'cash_MONTHS_BALANCE_cnt':'numerical_features', 'cash_CNT_INSTALMENT_min':'numerical_features', 'cash_CNT_INSTALMENT_max':'numerical_features', 'cash_CNT_INSTALMENT_set':'categorical_features', 'cash_CNT_INSTALMENT_FUTURE_min':'numerical_features', 'cash_CNT_INSTALMENT_FUTURE_max':'numerical_features', 'cash_NAME_CONTRACT_STATUS_cntd':'numerical_features', 'cash_NAME_CONTRACT_STATUS_set':'categorical_features', 'cash_NAME_CONTRACT_STATUS_latest':'categorical_features', 'cash_SK_DPD_max':'numerical_features', 'cash_SK_DPD_cnt0':'numerical_features', 'cash_SK_DPD_DEF_max':'numerical_features', 'cash_SK_DPD_DEF_cnt0':'numerical_features', 'cash_SK_DPD_diff_max':'numerical_features', 'cash_SK_DPD_diff_cnt0':'numerical_features', 
'credit_MONTHS_BALANCE_min':'numerical_features', 'credit_MONTHS_BALANCE_max':'numerical_features', 'credit_MONTHS_BALANCE_count':'numerical_features', 'credit_AMT_BALANCE_min':'numerical_features', 'credit_AMT_BALANCE_max':'numerical_features', 'credit_AMT_BALANCE_sum':'numerical_features', 'credit_AMT_BALANCE_mean':'numerical_features', 'credit_AMT_BALANCE_size':'numerical_features', 'credit_AMT_CREDIT_LIMIT_ACTUAL_min':'numerical_features', 'credit_AMT_CREDIT_LIMIT_ACTUAL_max':'numerical_features', 'credit_AMT_DRAWINGS_ATM_CURRENT_min':'numerical_features', 'credit_AMT_DRAWINGS_ATM_CURRENT_size':'numerical_features', 'credit_AMT_DRAWINGS_ATM_CURRENT_mean':'numerical_features', 'credit_AMT_DRAWINGS_ATM_CURRENT_sum':'numerical_features', 'credit_AMT_DRAWINGS_ATM_CURRENT_max':'numerical_features', 'credit_AMT_DRAWINGS_CURRENT_min':'numerical_features', 'credit_AMT_DRAWINGS_CURRENT_size':'numerical_features', 'credit_AMT_DRAWINGS_CURRENT_mean':'numerical_features', 'credit_AMT_DRAWINGS_CURRENT_sum':'numerical_features', 'credit_AMT_DRAWINGS_CURRENT_max':'numerical_features', 'credit_AMT_DRAWINGS_OTHER_CURRENT_min':'numerical_features', 'credit_AMT_DRAWINGS_OTHER_CURRENT_size':'numerical_features', 'credit_AMT_DRAWINGS_OTHER_CURRENT_mean':'numerical_features', 'credit_AMT_DRAWINGS_OTHER_CURRENT_sum':'numerical_features', 'credit_AMT_DRAWINGS_OTHER_CURRENT_max':'numerical_features', 'credit_AMT_DRAWINGS_POS_CURRENT_min':'numerical_features', 'credit_AMT_DRAWINGS_POS_CURRENT_size':'numerical_features', 'credit_AMT_DRAWINGS_POS_CURRENT_mean':'numerical_features', 'credit_AMT_DRAWINGS_POS_CURRENT_sum':'numerical_features', 'credit_AMT_DRAWINGS_POS_CURRENT_max':'numerical_features', 'credit_AMT_INST_MIN_REGULARITY_min':'numerical_features', 'credit_AMT_INST_MIN_REGULARITY_max':'numerical_features', 'credit_AMT_INST_MIN_REGULARITY_cntd':'numerical_features', 'credit_AMT_PAYMENT_CURRENT_sum':'numerical_features', 'credit_AMT_PAYMENT_CURRENT_max':'numerical_features', 'credit_AMT_PAYMENT_CURRENT_cntd':'numerical_features', 'credit_AMT_PAYMENT_TOTAL_CURRENT_sum':'numerical_features', 'credit_AMT_PAYMENT_TOTAL_CURRENT_max':'numerical_features', 'credit_AMT_PAYMENT_TOTAL_CURRENT_cntd':'numerical_features', 'credit_AMT_RECEIVABLE_PRINCIPAL_sum':'numerical_features', 'credit_AMT_RECEIVABLE_PRINCIPAL_max':'numerical_features', 'credit_AMT_RECEIVABLE_PRINCIPAL_cntd':'numerical_features', 'credit_AMT_RECIVABLE_sum':'numerical_features', 'credit_AMT_RECIVABLE_max':'numerical_features', 'credit_AMT_RECIVABLE_cntd':'numerical_features', 'credit_AMT_TOTAL_RECEIVABLE_sum':'numerical_features', 'credit_AMT_TOTAL_RECEIVABLE_max':'numerical_features', 'credit_AMT_TOTAL_RECEIVABLE_cntd':'numerical_features', 'credit_CNT_DRAWINGS_ATM_CURRENT_sum':'numerical_features', 'credit_CNT_DRAWINGS_ATM_CURRENT_max':'numerical_features', 'credit_CNT_DRAWINGS_ATM_CURRENT_mean':'numerical_features', 'credit_CNT_DRAWINGS_CURRENT_sum':'numerical_features', 'credit_CNT_DRAWINGS_CURRENT_max':'numerical_features', 'credit_CNT_DRAWINGS_CURRENT_mean':'numerical_features', 'credit_CNT_DRAWINGS_OTHER_CURRENT_sum':'numerical_features', 'credit_CNT_DRAWINGS_OTHER_CURRENT_max':'numerical_features', 'credit_CNT_DRAWINGS_OTHER_CURRENT_mean':'numerical_features', 'credit_CNT_DRAWINGS_POS_CURRENT_sum':'numerical_features', 'credit_CNT_DRAWINGS_POS_CURRENT_max':'numerical_features', 'credit_CNT_DRAWINGS_POS_CURRENT_mean':'numerical_features', 'credit_CNT_INSTALMENT_MATURE_CUM_cntd':'numerical_features', 'credit_CNT_INSTALMENT_MATURE_CUM_set':'categorical_features', 'credit_CNT_INSTALMENT_MATURE_CUM_latest':'categorical_features', 'credit_NAME_CONTRACT_STATUS_cntd':'categorical_features', 'credit_NAME_CONTRACT_STATUS_set':'categorical_features', 'credit_SK_DPD_max':'numerical_features', 'credit_SK_DPD_size':'numerical_features', 'credit_SK_DPD_DEF_max':'numerical_features', 'credit_SK_DPD_DEF_size':'numerical_features', 'credit_SK_DPD_diff_max':'numerical_features', 'credit_SK_DPD_diff_size':'numerical_features', 
'installment_NUM_INSTALMENT_VERSION_nunique':'numerical_features', 'installment_NUM_INSTALMENT_VERSION_set':'categorical_features', 'installment_NUM_INSTALMENT_VERSION_latest':'categorical_features', 'installment_NUM_INSTALMENT_NUMBER_min':'numerical_features', 'installment_NUM_INSTALMENT_NUMBER_max':'numerical_features', 'installment_NUM_INSTALMENT_NUMBER_count':'numerical_features', 'installment_NUM_INSTALMENT_NUMBER_nunique':'numerical_features', 'installment_DAYS_INSTALMENT_min':'numerical_features', 'installment_DAYS_INSTALMENT_max':'numerical_features', 'installment_DAYS_INSTALMENT_median':'numerical_features', 'installment_DAYS_INSTALMENT_mean':'numerical_features', 'installment_DAYS_INSTALMENT_range':'numerical_features', 'installment_DAYS_ENTRY_PAYMENT_min':'numerical_features', 'installment_DAYS_ENTRY_PAYMENT_max':'numerical_features', 'installment_DAYS_ENTRY_PAYMENT_median':'numerical_features', 'installment_DAYS_ENTRY_PAYMENT_mean':'numerical_features', 'installment_DAYS_ENTRY_PAYMENT_range':'numerical_features', 'installment_DPD_min':'numerical_features', 'installment_DPD_max':'numerical_features', 'installment_DPD_median':'numerical_features', 'installment_DPD_mean':'numerical_features', 'installment_DPD_sum':'numerical_features', 'installment_DPD_std':'numerical_features', 'installment_AMT_INSTALMENT_min':'numerical_features', 'installment_AMT_INSTALMENT_max':'numerical_features', 'installment_AMT_INSTALMENT_mean':'numerical_features', 'installment_AMT_INSTALMENT_sum':'numerical_features', 'installment_AMT_INSTALMENT_median':'numerical_features', 'installment_AMT_INSTALMENT_std':'numerical_features', 'installment_AMT_INSTALMENT_nunique':'numerical_features', 'installment_AMT_PAYMENT_min':'numerical_features', 'installment_AMT_PAYMENT_max':'numerical_features', 'installment_AMT_PAYMENT_mean':'numerical_features', 'installment_AMT_PAYMENT_sum':'numerical_features', 'installment_AMT_PAYMENT_median':'numerical_features', 'installment_AMT_PAYMENT_std':'numerical_features', 'installment_AMT_PAYMENT_nunique':'numerical_features', 'installment_AMT_DPD_min':'numerical_features', 'installment_AMT_DPD_max':'numerical_features', 'installment_AMT_DPD_mean':'numerical_features', 'installment_AMT_DPD_sum':'numerical_features', 'installment_AMT_DPD_median':'numerical_features', 'installment_AMT_DPD_std':'numerical_features', 'installment_AMT_DPD_nunique':'numerical_features',
}

In [12]:
numerical_features = [f for f in features_type.keys() if features_type[f] == 'numerical_features']
categorical_features = [f for f in features_type.keys() if features_type[f] == 'categorical_features']
amount_features = [f for f in features_type.keys() if features_type[f] == 'amount_features']
pivot_features = [f for f in features_type.keys() if features_type[f] == 'pivot_features']


def latest(group):
    return list(group)[0]

def countd(group):
    return group.nunique()

def collect_set(group):
    return ','.join(set(','.join(group).split(',')))

def cnt_positive(group):
    return sum(group>0)

def cnt_negative(group):
    return sum(group<0)


In [13]:
prev_source_features = pd.concat([
    prev[numerical_features].groupby(prev.SK_ID_CURR).agg(['min','max','mean','median',latest]),
    prev[categorical_features].astype(str).groupby(prev.SK_ID_CURR).agg([collect_set,countd,latest]),
    prev[pivot_features].astype(str).groupby(prev.SK_ID_CURR).agg([collect_set,countd,latest]),
    prev[amount_features].groupby(prev.SK_ID_CURR).agg(['sum','mean','max','min']),
], axis = 1)

prev_source_features.columns = ['_'.join(col) for col in prev_source_features.columns]


In [14]:
prev_special_features = pd.concat([
    prev.SK_ID_PREV.groupby(prev.SK_ID_CURR).agg('count'),
    prev.AMT_DOWN_PAYMENT.groupby(prev.SK_ID_CURR).agg([cnt_negative,cnt_positive]),
    prev.HOUR_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).agg('median'),
    prev.FLAG_LAST_APPL_PER_CONTRACT.groupby(prev.SK_ID_CURR).agg(lambda x: 1 if np.min(x) == 'Y' else 0),
    prev.NFLAG_LAST_APPL_IN_DAY.groupby(prev.SK_ID_CURR).agg('min'),
    prev.NFLAG_INSURED_ON_APPROVAL.groupby(prev.SK_ID_CURR).agg('min'),
    ], axis = 1)

prev_special_features.columns = ['PREV_cnt','AMT_DOWN_PAYMENT_cnt_negative','AMT_DOWN_PAYMENT_cnt_positive','HOUR_APPR_PROCESS_START_median','FLAG_LAST_APPL_PER_CONTRACT_1','NFLAG_LAST_APPL_IN_DAY_1','NFLAG_INSURED_ON_APPROVAL']


In [15]:
prev_features = pd.concat([
    prev_special_features,
    pd.get_dummies(prev[pivot_features + ['SK_ID_CURR']].set_index('SK_ID_CURR')).groupby(level=0).agg('max'),
    prev_source_features], axis = 1)
prev_features = prev_features.reset_index()

In [16]:
prev_features.shape
prev_features.head()

(338857, 842)

,SK_ID_CURR,PREV_cnt,AMT_DOWN_PAYMENT_cnt_negative,AMT_DOWN_PAYMENT_cnt_positive,HOUR_APPR_PROCESS_START_median,FLAG_LAST_APPL_PER_CONTRACT_1,NFLAG_LAST_APPL_IN_DAY_1,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,...,AMT_CREDIT_max,AMT_CREDIT_min,AMT_GOODS_PRICE_sum,AMT_GOODS_PRICE_mean,AMT_GOODS_PRICE_max,AMT_GOODS_PRICE_min,AMT_DOWN_PAYMENT_sum,AMT_DOWN_PAYMENT_mean,AMT_DOWN_PAYMENT_max,AMT_DOWN_PAYMENT_min
0,100001,1,0.0,1.0,13.0,1,1,0.0,1,0,...,23787.0,23787.0,24835.5,24835.5,24835.5,24835.5,2520.0,2520.0,2520.0,2520.0
1,100002,1,0.0,0.0,9.0,1,1,0.0,1,0,...,179055.0,179055.0,179055.0,179055.0,179055.0,179055.0,0.0,0.0,0.0,0.0
2,100003,3,0.0,1.0,15.0,1,1,0.0,1,0,...,1035882.0,68053.5,1306309.5,435436.5,900000.0,68809.5,6885.0,3442.5,6885.0,0.0
3,100004,1,0.0,1.0,5.0,1,1,0.0,1,0,...,20106.0,20106.0,24282.0,24282.0,24282.0,24282.0,4860.0,4860.0,4860.0,4860.0
4,100005,2,0.0,1.0,10.5,1,1,0.0,1,1,...,40153.5,0.0,44617.5,44617.5,44617.5,44617.5,4464.0,4464.0,4464.0,4464.0


In [17]:

prev_features.to_csv('./data/rawdata/prev_features.csv',index=False)
